#Textanalyse mit dem nltk-Package

### Vorbereitung

Zunächst verbinden wir das Notebook wieder mit dem Google-Drive, um auf Dateien zugreifen zu können

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Packages

Python eignet sich deshalb so gut für Textverarbeitung, weil es viele vorgefertigte Packages gibt, die man relativ einfach anwenden kann.

Diese Packages muss man zunächst vorinstallieren und dann mit dem Befehl `import` importieren



Zunächst installieren wir das `nltk`-Package, das viele Funktionen und Methoden zum Natural Language Processing bereithält, mit `pip`

In [ ]:
!pip install nltk

Dann importieren wir das Package:

In [ ]:
import nltk

... und noch ein Import

In [ ]:
from nltk.corpus.reader import CategorizedPlaintextCorpusReader

### Korpus anlegen

Mit `nltk` kann man einzelne Text-Files verarbeiten, aber auch gleich ein Korpus mit mehreren Texten anlegen.

Wir wollen die Minnesangtexte in eine Korpus importieren. Der Übersichtlichkeit halber packen wir den Pfadnamen zunächst in eine Variable

In [ ]:
path = '/content/drive/My Drive/DigMedTutorial/MS_nltk/'

Nun verwenden wir die Funktion `CategorizedPlaintextCorpusReader` zum Importieren, die nicht nur die Files importiert, sondern auch nach Mustern im Dateinamen zu Kategorien zusammenfasst.

Sowohl was importiert wird als auch die Kategorisierung erfolgt unter Zuhilfenahme von Regular Expressions (RegEx, wird im folgenden Beispiel durch das 'r' vor dem String angezeigt). Diese dienen zur Fuzzy-Suche und können sehr umfangreich sein, hier nur das Wichtigste bzw. die hier verwendeten Platzhalter:

* \.    beliebiges Zeichen
* \*    das vorangehende Zeichen kommt 0 oder mehrmals vor
* \+    das vorangehende Zeichen kommt mindestens ein- oder mehrmals vor
* \?    0 oder 1 Vorkommen
* \d    Eine Ziffer
* \     Das nachfolgende Zeichen ist buchstäblich zu lesen und setzt die 'Funktion als Platzhalter außer Kraft (mit \. kann man also einen Punkt adressieren, ohne den Schrägstrich wäre der ja beliebiges Zeichen)
* ()    Runde Klammern gruppieren einen Teil des Ausdrucks  




In [ ]:
ms_corpus = CategorizedPlaintextCorpusReader(path, r'.*\.txt', cat_pattern = r'\d_\d\d_(.*?)_.*')

Die Einzelfiles lassen sich mit der Methode fileids() aufrufen (im Folgenden beschränke ich die Ausgabe auf die ersten 20 Texte).

In [ ]:
ms_corpus.fileids()[:20]

Mit categories() werden die Kategorien ersichtlich, die schlicht die Autornamen sind.

In [ ]:
ms_corpus.categories()

Damit lassen sich nun einzelne Kategorien aufgreifen:

In [ ]:
ms_corpus.fileids(categories='Winli')

Mehrere Kategorien können mit einer Liste adressiert werden:

In [ ]:
ms_corpus.fileids(categories=['Winli','Steinmar'])

Mit len() kann die Länge der Liste an Texten zu einer Kategorie ausgegeben werden, also schlicht die Anzahl der Texte für eine Kategorie:

In [ ]:
len(ms_corpus.fileids(categories='Winli'))

Übung: Wie viele Lieder von Walther gibt es im Korpus?

## Statistische Auswertung

### Types und Tokens

Als erstes zählen wir schlicht, wie viele Wörter im Korpus vorkommen. In der Korpuslinguistik heißen Wörter Tokens.

Mit der Methode `words()` werden alle Wörter aus dem Korpus aufgegriffen, das packen wir in eine Variable, mit `len()` bestimmen wir, wie viele Wörter es sind (kann ein wenig dauern).



In [ ]:
tokens_ges = ms_corpus.words()
print(len(tokens_ges))

Sehen wir mal vorsichtig in tok_ges rein:

In [ ]:
tokens_ges[:20]

Hier sind noch Satzzeichen drin, die wollen wir draußen haben.

Dazu können wir die Methode `isalpha()` verwenden und zwar in einer so genannten list comprehension: Wir iterieren über die Liste und nehmen nur jene Einträge, für die gilt, dass sie aus Buchstaben bestehen:



In [ ]:
tokens_ges_op = [w for w in tokens_ges if w.isalpha()]

Übung:

Die Anzahl der Tokens ohne Satzzeichen ist daher:

Types sind die unikalen Wortformen in einem Text. Der Satz "ich bin ich" enthält beispielsweise 3 Tokens (ich, bin, ich), aber nur 2 Types (ich, bin)

Gleiche Listenpunkte lassen sich in Python einfach mit der Funktion `set()` gruppieren


In [ ]:
types_ges_op = set(tokens_ges_op)


Aber Achtung, hier werden Wörter, die manchmal groß, manchmal klein geschrieben sind, als unterschiedlich behandelt.

Daher verwandeln wir vor Anwendung von set() besser alles in Kleinbuchstaben, wieder mit einer List comprehension:


In [ ]:
types_ges_op = set(w.lower() for w in tokens_ges_op)

In [ ]:
print(len(types_ges_op))

#### Type-Token-Ratio

Ein Standardmaß für die Beschreibung der lexikalischen Dichte ist die Type-Token-Ratio, bei der die Anzahl der Tokens zur Anzahl der Types in Beziehung gesetzt wird.

In [ ]:
ttr = len(types_ges_op) / len(tokens_ges_op)
print(ttr)

Die Type-Token-Ratio misst also, wie "komplex" ein Text hinsichtlich seines Sprachgebrauchs ist.

Das Maß hat aber einen Nachteil: Die TTR ist abhängig von der Länge eines Textes (je länger ein Text ist, desto eher wird er dieselben Wörter benutzen, die TTR sinkt also automatisch).

Es gibt daher ein paar verfeinerte Methoden, mit der man das Maß normieren kann. Beim "Measure of Textual Lexical Diversity (MTLD)" wird die durchschnittliche Länge der Abschnitte gemessen, die über einen bestimmten Schwellwert der TTR liegen.

Zur Berechnung der MTLD verwenden wir ein weiteres Package, nämlich LexicalRichness.

In [ ]:
!pip install lexicalrichness
from lexicalrichness import LexicalRichness


Um das Package zu verwenden, müssen wir die Wort-Liste wieder in eine einzige Textvariable zurückführen, machen wir mit join() vor (join() steht das Trennzeichen, das eingefügt wird, wenn die Listenpunkte zusammengeführt werden. ' '.join() setzt also schlicht vor jedes Wort ein Leerzeichen.

In [ ]:
tokens_ges_op_collapsed = ' '.join(tokens_ges_op)

Darauf können wir jetzt die Funktion LexicalRichness auführen, die alles berechnet:

In [ ]:
lex = LexicalRichness(tokens_ges_op_collapsed)

In [ ]:
print("Tokens: ",lex.words, "Types: ",lex.terms, "TTR: ",lex.ttr)

In [ ]:
print("MTLD: ", lex.mtld(threshold=0.72))

### Statistiken für Einzeltexte:

In [ ]:
for fileid in ms_corpus.fileids():
  tokens_op = [w for w in ms_corpus.words(fileid) if w.isalpha]
  num_words = len(tokens_op)
  num_types = len(set(w.lower() for w in tokens_op))

  print("Text ",fileid, " hat ", num_words, " Tokens und ", num_types, "Types")

Übung: Wie sieht das Ganze für die Kategorien aus?

#### Dataframe

Wir machen einen Dataframe (eine Art Tabelle) mit Kennzahlen für alle Kategorien:

Wir brauchen dafür zwei neue Packages, die weit verbreiteten numpy und pandas-Packages zur Datenstrukturierung.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## Wir beginnen mit einer leeren Liste:

a=list()


## ... füllen eine Zeile mit den Informationen

for cats in ms_corpus.categories():


  ### Anzahl der Zeichen

  text_char = [w.lower() for w in ms_corpus.raw(categories=cats) if w.isalpha()]
  num_chars = len(text_char)


  ### Tokenanzahl
  words_op = [w.lower() for w in ms_corpus.words(categories=cats) if w.isalpha()]
  num_words = len(words_op)

  ### Durchschnittliche Wortlänge
  av_wordlen = round(num_chars/num_words,2)

  ### Anzahl Types

  num_types = len(set(w.lower() for w in words_op))

  ### MTLD

  words_op_collapsed = ' '.join(words_op)
  lex = LexicalRichness(words_op_collapsed)
  mtld_ind = lex.mtld(threshold=0.72)


  ### ... und fügen die Zeile unserer Tabelle hinzu

  a.append(np.array([cats, int(num_chars),int(num_words),int(num_types),int(mtld_ind), int(av_wordlen) ]))

a = np.asarray(a)

Wir konvertieren die Tabelle in einen Dataframe und vergeben Spaltennamen:

In [ ]:
df = pd.DataFrame(a,columns=['Autor','Zeichen','Tokens','Types','Mtld','Wortlänge'])

In [ ]:
df['Zeichen'] = df['Zeichen'].astype(int)
df['Tokens'] = df['Tokens'].astype(int)
df['Types'] = df['Types'].astype(int)
df['Mtld'] = df['Mtld'].astype(int)
df['Wortlänge'] = df['Wortlänge'].astype(float)

In [ ]:
df

Der Dataframe lässt sich leicht sortieren

In [ ]:
df.sort_values('Tokens',ascending=False)

Übung: Welche Autoren haben die höchste lexikalische Diversität?

### Visualisierung

Wir brauchen zwei weitere Packages



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Wir sortieren den Dataframe wie vorher und nehmen nur die 20 Bestplatzierten

In [ ]:
df_sort = df.sort_values('Tokens',ascending=False)[:20]

In [ ]:
plt.figure(figsize=(15,10))
bp = sns.barplot(x="Tokens", y="Autor", data=df_sort, edgecolor=".3",
            linewidth=0.5,)
bp.set(xlabel='Mtld', ylabel='Texte', title='Verteilung Textlängen')
plt.savefig('TextenachTokens_bar.png')

### Wortfrequenzen

Sind leicht zu berechnen, mit der Funktion `FreqDist` wird ein frequence table erstellt, der die Zählung für jedes Wort enthält


In [ ]:
fdist_ges = nltk.FreqDist(w.lower() for w in tokens_ges)

Aus dem Frequence Table können leicht die Werte für einzelne Wörter ausgelesen werden:

In [ ]:
print(fdist_ges['leit'])

Oft wird es sinnvoll sein, nicht die absolute Frequenz, sondern die relative Frequenz zu bestimmen, also die Anzahl der Belege geteilt durch die Gesamtlänge des Textes:

In [ ]:
print(fdist_ges.freq('leit'))

Die Frequenz für mehrere Wörter bestimmen...

In [ ]:
wortliste = ['tugende','êre','mâze','zuht','muot','triuwe','schame','vuoge']
freqliste = []


for w in wortliste:
    print(w, ': ', fdist_ges[w])
    freqliste.append(fdist_ges[w])

... und visualisieren:

In [ ]:
plt.bar(wortliste, freqliste)
plt.xticks(wortliste, rotation=45)
plt.title('Belege Gesamtkorpus')
plt.xlabel('Wörter')
plt.ylabel('Belege')
plt.show()

Was sind die (zwanzig) häufigsten Wörter im Gesamtkorpus?

In [ ]:
fdist_ges.most_common(20)

Satzzeichen sollten raus!

In [ ]:
fdist_ges_op = nltk.FreqDist(w.lower() for w in tokens_ges_op)

In [ ]:
fdist_ges_op.most_common(20)

Die häufigsten Wörter in Texten sind Funktionswörter! Und zwar überproportional häufig -> Zipfs Gesetz

In [ ]:
wortliste = [w[0] for w in fdist_ges_op.most_common(100)]
freqliste = [w[1] for w in fdist_ges_op.most_common(100)]




In [ ]:
plt.bar(wortliste, freqliste)
plt.xticks(wortliste, rotation=90)
plt.title('MFW Gesamtkorpus')
plt.xlabel('Wörter')
plt.ylabel('Belege')
plt.show()

### Funktionswörter ausschließen mit Stoppwortliste

Zunächst muss die Stoppwortliste importiert werden:

In [ ]:
filepath = '/content/drive/My Drive/DigMedTutorial/Stoplist_GMH.txt'
f=open(filepath,'r', encoding='utf8')
stopwords=f.read().splitlines()



Sieht so aus:

In [ ]:
stopwords[:15]

Stoppwörter ausschließen:

In [ ]:
tokens_ges_op_osw = [w for w in tokens_ges_op if w not in stopwords]

Neuer Frequency Table:

In [ ]:
fdist_ges_op_osw = nltk.FreqDist(w.lower() for w in tokens_ges_op_osw)

In [ ]:
fdist_ges_op_osw.most_common(20)

### Frequency Table für alle Autoren machen:





In [ ]:
freq_dist = {}

for cats in ms_corpus.categories():
  text_li = []
  for fileid in ms_corpus.fileids(categories=cats):
    text = list(ms_corpus.words(fileids=fileid))
    text_li = text_li + text

  freq_dist[cats] = nltk.FreqDist([w.lower() for w in text_li])

freq_dist ist ein sogenanntes Dictionary, auf das mit dem Key (hier einfach der Autorname) zurückgegriffen werden kann:



In [ ]:
freq_dist['Walther'].most_common(20)

In [ ]:
freq_dist['Walther'].freq('leit')

## Kollokationen im Text finden

Kollokationen sind Wörter, die gemeinsam im Text auftreten (in einem bestimmten Kontext-Window, siehe Parameter `window_size`)

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

Kollokationsliste erstellen:

In [ ]:
words = [w.lower() for w in tokens_ges]

bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(words, window_size=2)

bigramFinder.apply_word_filter(lambda w: not w.isalpha())

... und sortieren nach häufigster Kollokation:

In [ ]:
bigramratio_ges = bigramFinder.score_ngrams(bigram_measures.raw_freq)

In [ ]:
bigramratio_ges[:20]

Die Ergebnisse sind wenig aussagekräftig, da Wörter, die häufig auftreten, auch oft kookurrieren.

Abhilfe: Ranking der Liste nach Likelihood-Maß, das Überzufälligkeit der Kollokation berücksichtigt.

In [ ]:
bigramratio_ges = bigramFinder.score_ngrams(bigram_measures.likelihood_ratio)

In [ ]:
bigramratio_ges[:20]

Schließlich noch möglich: Ausschluss von Stoppwörtern:

In [ ]:
words = [w.lower() for w in tokens_ges]

bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(words, window_size=2)

bigramFinder.apply_word_filter(lambda w: not w.isalpha() or w in stopwords )

In [ ]:
bigramratio_ges[:20]